# Fit bmimpf6 data

EoS: PR

Parameter T dependence: Linear

N (total fitting parameters): 4

## Import Functions

In [1]:
import idaes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize

import sys
sys.path.append('../../../')

from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)
from pyomo.opt import TerminationCondition, SolverStatus

from idaes.core import FlowsheetBlock
import idaes.logger as idaeslog
# Import the Generic Parameter Block
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)
# Import unit models from the model library
from idaes.generic_models.unit_models import Flash
# Import degrees of freedom tool
from idaes.core.util.model_statistics import degrees_of_freedom

# parmest (binary_param2)
from bip_fitting_functions import linear

import scipy.stats as stats

## Load Data

In [2]:
data_full = pd.read_csv('../r32_bmimpf6_full.csv')
data_subset = pd.read_csv('../r32_bmimpf6_subset.csv')
data_extended = pd.read_csv('../r32-bmimpf6-extended.csv')

## Load Configuration + Check DOF + Load Parameters

In [3]:
lhs_results1 = pd.read_csv('../Data/Fits/LHS_Fits/PR_linTdep_LHS.csv',index_col=0)
lhs_results1 = lhs_results1[['sc_param1','sc_param2','sc_param3','sc_param4','SSR']]

lhs_results2 = pd.read_csv('Data/Init_Final/LHS_Output/PR_linTdep_LHS.csv',index_col=0)
lhs_results2 = lhs_results2[['sc_param1','sc_param2','sc_param3','sc_param4','SSR']]

lhs_results = pd.concat([lhs_results1,lhs_results2],axis=0).reset_index(drop=True)

In [4]:
lhs_results.dropna(axis=0)

,sc_param1,sc_param2,sc_param3,sc_param4,SSR
26,-0.375036,-0.440945,0.555141,0.413887,2.503494
70,-0.495915,0.312509,0.820636,-0.314104,1.507121
189,0.596881,-0.709315,-0.383858,0.762490,3.995125
505,-0.772163,-0.032215,0.612869,-0.001824,0.160256
830,-1.051539,-1.698288,1.620456,1.573882,3.676243
2308,0.946113,-1.782356,-1.080670,1.838949,0.134513
2509,0.905605,-1.038117,-0.917062,1.022360,3.265507
4301,-1.570062,0.451052,1.651695,-0.439320,0.134513
4559,-0.365087,-0.492002,0.335834,0.456671,3.265507
4635,0.954985,-0.751578,-0.353782,0.744256,6.826056


In [5]:
lhs_best = lhs_results[lhs_results.SSR == lhs_results.SSR.min()]
lhs_best

,sc_param1,sc_param2,sc_param3,sc_param4,SSR
2308,0.946113,-1.782356,-1.08067,1.838949,0.134513


In [6]:
sc_param1 = lhs_results.at[2308, 'sc_param1']
sc_param2 = lhs_results.at[2308, 'sc_param2']
sc_param3 = lhs_results.at[2308, 'sc_param3']
sc_param4 = lhs_results.at[2308, 'sc_param4']

In [7]:
from hfc32_bmimpf6_PR import configuration 

In [8]:
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})
m.fs.properties = GenericParameterBlock(default=configuration)
m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})
DOF_initial = degrees_of_freedom(m)
print('The initial DOF is {0}'.format(DOF_initial))

The initial DOF is 7


## Fit Parameters

In [9]:
parameters, obj_value, a = linear(data_subset, configuration, 'R32', 'bmimpf6', "x_R32", "x_bmimpf6", 
        init_temp =  283.1, init_press =   399300 , init_x_c1 =    0.448, init_x_c2 = 0.552,
        init_kappa_2_1A = sc_param1, init_kappa_1_2A = sc_param2,
        init_kappa_2_1B = sc_param3, init_kappa_1_2B = sc_param4, 
        eps = 0.1, scaling_fac = 1e-9, 
        filename='Data/Fits/Ipopt_Output/PR_linTdep_final.txt')

2022-09-19 12:11:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-09-19 12:11:37 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-09-19 12:11:37 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-09-19 12:11:37 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-09-19 12:11:38 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-09-19 12:11:38 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-09-19 12:11:39 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-09-19 12:11:39 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-09-19 12:11:40 [INFO] idaes

/Users/bridgettebefort/opt/anaconda3/envs/idaes-pse-dev/lib/python3.9/site-packages/pyomo/contrib/interior_point/interface.py:418: RuntimeWarning: divide by zero encountered in true_divide
  data = (duals_primals_lb/(primals - self._nlp.primals_lb()) +


In [10]:
print('Objective (SSE):',obj_value)

Objective (SSE): 0.13451314174278697


In [11]:
print('Parameters:',parameters)

Parameters: fs.properties.PR_kappa_A[R32,bmimpf6]    0.027876
fs.properties.PR_kappa_A[bmimpf6,R32]    0.797811
fs.properties.PR_kappa_B[R32,bmimpf6]   -0.057162
fs.properties.PR_kappa_B[bmimpf6,R32]   -0.872324
dtype: float64


In [12]:
np.savetxt('Data/Fits/PR_params_linTdep.csv',parameters,delimiter=',')

In [13]:
cov = np.array(a)
print('Covariance Matrix:')
print(cov)

Covariance Matrix:
[[ 0.00015585  0.00129564 -0.00014419 -0.0012408 ]
 [ 0.00129564  0.05597866 -0.00145117 -0.05826367]
 [-0.00014419 -0.00145117  0.00013546  0.0014213 ]
 [-0.0012408  -0.05826367  0.0014213   0.06081508]]


In [14]:
np.savetxt('Data/Covariance/PR_Cov_linTdep.csv',cov,delimiter=',')

## Apply Parameters + Plot

In [ ]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [ ]:
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False})

m.fs.properties = GenericParameterBlock(default=configuration)

m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

m.fs.F101.inlet.flow_mol.fix(1)
m.fs.F101.inlet.temperature.fix(283.15)
m.fs.F101.inlet.pressure.fix(10000)
m.fs.F101.inlet.mole_frac_comp[0,'R32'].fix(0.05)

m.fs.properties.PR_kappa_A['R32','bmimpf6'] = parameters['fs.properties.PR_kappa_A[R32,bmimpf6]']
m.fs.properties.PR_kappa_A['bmimpf6','R32'] = parameters['fs.properties.PR_kappa_A[bmimpf6,R32]']
m.fs.properties.PR_kappa_B['R32','bmimpf6'] = parameters['fs.properties.PR_kappa_B[R32,bmimpf6]']
m.fs.properties.PR_kappa_B['bmimpf6','R32'] = parameters['fs.properties.PR_kappa_B[bmimpf6,R32]']

# Initialize the flash unit
m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)

# Fix the state variables on the state block
# m.fs.F101.inlet.pressure.unfix()
m.fs.F101.inlet.mole_frac_comp[0,'bmimpf6'].unfix()

R32_x = np.zeros((len(data_extended)))
P = np.zeros((len(data_extended)))
T = np.zeros((len(data_extended)))

m.fs.liq = Param(mutable=True,default=0.040)
m.fs.liquid = Constraint(expr=m.fs.F101.liq_outlet.mole_frac_comp[0, "bmimpf6"] == m.fs.liq)

for i in range(len(data_extended)):
    print('i:',i)
    print('Try x_IL:',data_extended["x_bmimpf6"].iloc[i])
    m.fs.liq = data_extended["x_bmimpf6"].iloc[i]
    m.fs.F101.inlet.temperature.fix(float(data_extended["temperature"].iloc[i]))
    m.fs.F101.inlet.pressure.fix(float(data_extended["pressure"].iloc[i]))
    m.fs.F101.inlet.mole_frac_comp[0,'R32'].fix(float(data_extended["x_R32"].iloc[i])+0.1)
    m.fs.F101.inlet.mole_frac_comp[0,'bmimpf6'].fix(float(1-(data_extended["x_R32"].iloc[i]+0.1)))
    m.fs.F101.vap_outlet.temperature.fix(float(data_extended["temperature"].iloc[i]))
    
    try:
        m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)
        
    except:
        print('Initialization Error')
        
    try:

        status = solver.solve(m, tee = False)
        
        if (status.solver.status == SolverStatus.ok) and (status.solver.termination_condition == TerminationCondition.optimal):
            print('Feasible/Optimal Solution')
            R32_x[i] = value(m.fs.F101.liq_outlet.mole_frac_comp[0,'R32']) 
            P[i] = value(m.fs.F101.vap_outlet.pressure[0])
            T[i] = data_extended["temperature"].iloc[i]
        else:
            print('Infeasible Solution')
            R32_x[i] = np.nan
            P[i] = np.nan
            T[i] = data_extended["temperature"].iloc[i]
            
    except ValueError:
        R32_x[i] = np.nan
        P[i] = np.nan
        T[i] = data_extended["temperature"].iloc[i]
        print('Solver Error')

In [ ]:
fitted_data = pd.DataFrame(data=[R32_x,P/1e6,T])
fitted_data = fitted_data.T
fitted_data.columns = ['R32_x','P, MPa','T, K']
fitted_data.to_csv('Data/Fits/PR_FittedData_linTdep.csv')

In [ ]:
plt.plot(fitted_data.where(fitted_data['T, K']==283.2)['R32_x'],fitted_data.where(fitted_data['T, K']==283.2)['P, MPa'],"b-",label='T=283 K')
plt.plot(fitted_data.where(fitted_data['T, K']==298.1)['R32_x'],fitted_data.where(fitted_data['T, K']==298.1)['P, MPa'],"r-",label='T=298 K')
plt.plot(fitted_data.where(fitted_data['T, K']==323.1)['R32_x'],fitted_data.where(fitted_data['T, K']==323.1)['P, MPa'],"c-",label='T=323 K')
plt.plot(fitted_data.where(fitted_data['T, K']==348.1)['R32_x'],fitted_data.where(fitted_data['T, K']==348.1)['P, MPa'],"m-",label='T=348 K')
plt.plot(data_full["x_R32"],data_full["pressure"]/1e6,"*",color='orange',label="data")  
plt.title('Isotherm R32-[bmim][pf6]')
plt.ylabel('Pressure (MPa)')
plt.xlabel('x R32')
plt.grid(True)
plt.ylim(0,1.110000)
plt.legend()
plt.show()

### Print Eigvals, Eigvectors

An identifiability analysis can be performed with these as well as a FIM because the covariance matrix is the inverse of the FIM. That means that if there are any eigvals > 0, their eigvecs indicate directions of un-identifiability.

In [ ]:
eigvals, eigvecs = np.linalg.eig(cov)

In [ ]:
eigvals

In [ ]:
eigvecs

In [ ]:
np.savetxt('Data/EigenAnalysis/PR_vals_linTdep.csv',eigvals,delimiter=',')
np.savetxt('Data/EigenAnalysis/PR_vecs_linTdep.csv',eigvecs,delimiter=',')

In [ ]:
FIM = np.linalg.inv(cov) 

#A_opt
FIM_trace = np.trace(FIM)
cov_trace = np.trace(cov)

#D_opt
FIM_det = np.linalg.det(FIM)

#E_opt
eigvals_FIM, eigvecs_FIM = np.linalg.eig(FIM)
eigvals_FIM_min = eigvals_FIM.min()
eigvals_FIM_max = eigvals_FIM.max()

np.savetxt('Data/EigenAnalysis/PR_FIMtrdet_linTdep.csv',[cov_trace,FIM_trace,FIM_det,eigvals_FIM_min,eigvals_FIM_max])
np.savetxt('Data/EigenAnalysis/PR_FIMvals_linTdep.csv',eigvals_FIM,delimiter=',')
np.savetxt('Data/EigenAnalysis/PR_FIMvecs_linTdep.csv',eigvecs_FIM,delimiter=',')

## AIC Value

See: https://doi.org/10.1080/21642583.2018.1496042

AIC(k) = Nln(MSE(k))+2k

N: number of data points

MSE(k): mean squared error of the model with k parameters

k: number of parameters

MSE(k) = SSE(k)/N

SSE: sum of squared error of the model with k parameters

In [ ]:
#number of datapoints
N = len(data_subset)
#MSE(k)
MSE = obj_value/N
#number of parameters
k = 4

In [ ]:
AIC = N*np.log(MSE) + 2*k
AIC

In [ ]:
AIC_data = pd.DataFrame(data=[N,MSE,k,obj_value,AIC])
AIC_data = AIC_data.T
AIC_data.columns = ['N','MSE','k','Objective, Pa^2','AIC']
AIC_data.to_csv('Data/AIC/PR_AIC_linTdep.csv')